In [2]:
#import the libraries as needed
import os
import re
import time
import csv
import random
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from datetime import date, timedelta, datetime
from urllib.request import urlopen
import pandas as pd
import numpy as np
from time import sleep
from random import randint
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent

In [2]:
! pip install pymysql

In [3]:
! pip install fake-useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=57eab433417992364fab7589e71b0ea8d820e6506cd06f86c3be8d334a76f491
  Stored in directory: c:\users\rcjan\appdata\local\pip\cache\wheels\a0\b8\b7\8c942b2c5be5158b874a88195116b05ad124bac795f6665e65
Successfully built fake-useragent


# Start of Important Code

In [60]:
#Reset list of URL's
correct_url = []
Url_list = []

In [158]:
#Reset Tables
ticker_data = []
company_data = []
title_data = []
date_data = []
time_data = []
quarter_data = []
company_info_table = []
Speech_table = []
QA_table = []
Comp_part_table = []
speech = []
speaker = []
QA_speech = []
QA_speaker = []
comp_parts = []
comp_pos = []
conf_parts = []
conf_pos = []
Conf_part_table = []
comp_ccid = []
speech_ccid = []
QA_ccid = []
comppart_ccid = []
confpart_ccid = []
speech_scriptid = []
QA_scriptid = []
comppart_partid = []
confpart_partid = []
test_list = []

In [161]:
def pullpageurls(siteurl):
    pages = np.arange(150,200,1)
    driver = webdriver.Chrome('./chromedriver')
    #Pulling seeking alpha transcript search page
    for page in pages:
        page = siteurl+"/"+str(page)
        driver.get(page)
        site_html = driver.page_source
        soup_site = BeautifulSoup(site_html,'html.parser')
        sleep(random.randint(1,5))
        #Pulling transcript urls from seeking alpha transcript search page
        url_begin = 'https://seekingalpha.com'
        site_content = soup_site.find_all('h3')
        #Extracting url and placing into a list
        for url in site_content:
            url_pull = url.find('a',class_ = 'dashboard-article-link')
            partial_url = url_pull.get('href')
            pageURL = url_begin+partial_url
            Url_list.append(pageURL)
    #Extracting only Url's with 2019 or 2020 in them
def pullpagedetails(Url_list):
    driver = webdriver.Chrome('./chromedriver')
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
    try:
        counter = company_info_table['CC ID'].iloc[-1] + 1
        counter2 = QA_table['Script ID'].iloc[-1] + 1
        counter3 = Conf_part_table['Part ID'].iloc[-1] + 1
    except:
        counter = 0
        counter2 = 0
        counter3 = 0
    for i in Url_list:
        if ('2020' in i or '2019'in i) and ('earnings' in i):
            #Pulling relevant information from URL's
            #Header to mimic browser visit
            #Create requests.models.response object from URL
            driver.get(i)
            html = driver.page_source
            #Create the soup
            soup = BeautifulSoup(html, "lxml")
            call = soup.find('div',id = 'content-rail')
            #Testing if URL contains necessary elements
            test_list = []
            test = call.find_all('p',class_ = re.compile('p p*'))
            for c in test:
                x = c.text.strip()
                test_list.append(x)
            if ('Operator' in test_list) and ('Conference Call Participants' in test_list):
                #Basic Company Info Section
                #Create cc_id
                cc_id = counter
                #Find company ticker
                try:
                    ticker = call.find('a',class_ = 'ticker-link').text.split(" ")[-1]
                    ticker_data.append(ticker)
                except:
                    ticker_data.append('None')
                #Find company name
                try:
                    company = call.find('a', class_ = 'ticker-link').text.split(" ")[0:-1]
                    company = " ".join(company)
                    company_data.append(company)
                except:
                    company_data.append('None')
                #Find call title
                try:
                    title = call.find('h1').text
                    title_data.append(title)
                except:
                    title_data.append('None')
                #Find quarter/year
                try:
                    date_text = call.find('p',class_ = 'p p1').text.split(') ')[1]
                    quarter = date_text[0:7]
                    quarter_data.append(quarter)
                except:
                    quarter_data.append('None')
                #Find call date
                try:
                    date_string = date_text[date_text.index('Call')+5:].split("Call ")[0]
                    date = date_string[:date_string.index(":")-2].strip()
                    date_data.append(date)
                except:
                    date_data.append('None')
                #Find call time
                try:
                    time = date_string[date_string.index(":")-2:].strip()
                    time_data.append(time)
                except:
                    time_data.append('None')
                #Add CC ID
                comp_ccid.append(cc_id)
        #Speech and Company Participants Section
                bj = []
                indices = []
                convo = []
                #Pull speech script section
                script = call.find_all('p',class_ = re.compile('p p*'))
                #pull strong tags from speech script section
                for i in script:
                    if i.find('strong') is not None:
                        g = "strong"
                    else:
                        g = 0
                    bj.append(g)
                #Create list of index position of strong tags
                for j in range(len(bj)):
                    if bj[j] == 'strong':
                      indices.append(j)
                #Create list of text from speech script
                for k in script:
                    x = k.text.strip()
                    convo.append(x)
                #Create script ID & Participant ID
                script_id = counter2
                part_id = counter3
                #Create index objects to find positions of speech & participant sections
                conf_part_index = convo[convo.index('Conference Call Participants')+1:convo.index('Operator')]
                try:
                    speech_index = indices[0:indices.index(convo.index('Question-and-Answer Session'))]
                except:
                    speech_index = indices[0:indices.index(convo.index('Question-And-Answer Session'))]
                try:
                    QA_index = indices[indices.index(convo.index('Question-and-Answer Session'))+1:]
                except:
                    QA_index = indices[indices.index(convo.index('Question-And-Answer Session'))+1:]
                try:
                    comp_part_index = convo[convo.index('Company Participants')+1:convo.index('Conference Call Participants')]
                except:
                    comp_part_index = convo[convo.index('Corporate Participants')+1:convo.index('Conference Call Participants')]
                #Pulling info from speech section
                for i in range(2,len(speech_index)-1):
                    try:
                        z = convo[speech_index[i]+1:speech_index[i+1]]
                        speech.append(z)
                    except:
                        speech.append('None')
                    try:
                        g = convo[speech_index[i]]
                        speaker.append(g)
                    except:
                        speaker.append('None')
                    #Add CC ID & Script ID
                    speech_ccid.append(cc_id)
                    speech_scriptid.append(script_id)
                counter2 += 1
                script_id = counter2
                #Pulling info from Q&A section
                for i in range(0,len(QA_index)-1):
                    try:
                        q = convo[QA_index[i]+1:QA_index[i+1]]
                        QA_speech.append(q)
                    except:
                        QA_speech.append('None')
                    try:
                        f = convo[QA_index[i]]
                        QA_speaker.append(f)
                    except:
                        QA_speaker.append('None')
                    #Add CC ID & Script ID
                    QA_ccid.append(cc_id)
                    QA_scriptid.append(script_id)
                counter2 += 1
                script_id = counter2
                #Pulling company participants
                for i in comp_part_index:
                    x = i.split('-')
                    try:
                        a = x[0]
                        comp_parts.append(a)
                    except:
                        comp_parts.append('None')
                    try:
                        b = x[1]
                        comp_pos.append(b)
                    except:
                        comp_pos.append('None')
                    #Add CC ID & Part ID
                    comppart_ccid.append(cc_id)
                    comppart_partid.append(part_id)
                    counter3 += 1
                    part_id = counter3
                #Pulling conference participants
                for i in conf_part_index:
                    x = i.split('-')
                    try:
                        a = x[0]
                        conf_parts.append(a)
                    except:
                        conf_parts.append('None')
                    try:
                        b = x[1]
                        conf_pos.append(b)
                    except:
                        conf_pos.append('None')
                    #Add CC ID & Part ID
                    confpart_ccid.append(cc_id)
                    confpart_partid.append(part_id)
                    counter3 += 1
                    part_id = counter3
                #Add to CC ID
                counter += 1
            else:
                continue
        #Pause in between trying each call transcript
        sleep(random.randint(1,2))



In [93]:
#Pull page urls
pullpageurls('https://seekingalpha.com/earnings/earnings-call-transcripts')

In [94]:
len(Url_list)

9360

In [172]:
take1 = Url_list[0:300]
take2 = Url_list[325:600]
take3 = Url_list[300:449]
take4 = Url_list[450:599]
take5 = Url_list[600:749]
take6 = Url_list[750:899]
take7 = Url_list[900:1049]
take8 = Url_list[1050:1199]
take9 = Url_list[1200:1349]
take10 = Url_list[1350:1499]

In [174]:
#Pull page details
pullpagedetails(take2)

In [216]:
#Create tables

#Create company info table
company_info_table = pd.DataFrame({'CC ID':comp_ccid,'Tickers':ticker_data,'Company':company_data,'Call Title':title_data,'Fiscal Quarter':quarter_data,'Date':date_data,'Time':time_data})
#Create Speech table
Speech_table = pd.DataFrame({'CC ID':speech_ccid,'Script ID':speech_scriptid,'Participant':speaker,'Speech':speech,'Section':'Speech'})
#Create Q&A table
QA_table = pd.DataFrame({'CC ID':QA_ccid,'Script ID':QA_scriptid,'Participant':QA_speaker,'Speech':QA_speech,'Section':'Q&A'})
#Create Company participant table
Comp_part_table = pd.DataFrame({'CC ID':comppart_ccid,'Part ID':comppart_partid,'Participant':comp_parts,'Position':comp_pos,'Organization':'None','Participant Type':'Internal'})
#Create Conference participant table
Conf_part_table = pd.DataFrame({'CC ID':confpart_ccid,'Part ID':confpart_partid,'Participant':conf_parts,'Position':'None','Organization':conf_pos,'Participant Type':'External'})

In [ ]:
#Save tables to Excel for future use

#path = r'C:\Users\rcjan\Documents\DataMgmtTables.xlsx'

#writer = pd.ExcelWriter(path, engine = 'xlsxwriter')
#company_info_table.to_excel(writer, sheet_name = 'Comp_info')
#Speech_table.to_excel(writer, sheet_name = 'Speech_info')
#QA_table.to_excel(writer, sheet_name = 'QA_info')
#Comp_part_table.to_excel(writer, sheet_name = 'Comp_part')
#Conf_part_table.to_excel(writer, sheet_name = 'Conf_part')

#writer.save()
#writer.close()

# Start Here:

In [14]:
#Pull in data from Excel file

#Put your filepath here
file = r'C:\Users\saman\Documents\Data Management\DataMgmtTables2.xlsx'

company_info_table = pd.read_excel(file,sheet_name='Comp_info')
Speech_table = pd.read_excel(file, sheet_name = 'Speech_info')
QA_table = pd.read_excel(file, sheet_name = 'QA_info')
Comp_part_table = pd.read_excel(file, sheet_name = 'Comp_part')
Conf_part_table = pd.read_excel(file, sheet_name = 'Conf_part')


In [15]:
len(company_info_table)

473

In [16]:
#Completed table of company info
company_info_table.tail(20)

,Unnamed: 0,CC ID,Tickers,Company,Call Title,Fiscal Quarter,Date,Time
453,453,441,(GTS),Triple-S Management Corporation,Triple-S Management Corporation (GTS) CEO Bobb...,Q3 2020,"November 6, 2020",8:30 AM ET
454,454,442,(EYPT),"EyePoint Pharmaceuticals, Inc.","EyePoint Pharmaceuticals, Inc.'s (EYPT) CEO Na...",Q3 2020,"November 5, 2020",8:30 AM ET
455,455,443,(AHH),"Armada Hoffler Properties, Inc.","Armada Hoffler Properties, Inc. (AHH) CEO Loui...",Q3 2020,"November 5, 2020",8:30 AM ET
456,456,444,(EVH),"Evolent Health, Inc.","Evolent Health, Inc. (EVH) CEO Seth Blackley o...",Q3 2020,"November 5, 2020",5:00 PM ET
457,457,445,(NCSM),"NCS Multistage Holdings, Inc.","NCS Multistage Holdings, Inc. (NCSM) CEO Rober...",Q3 2020,"November 5, 2020",8:30 AM ET
458,458,446,(CYRX),"Cryoport, Inc.","Cryoport, Inc. (CYRX) CEO Jerrell Shelton on Q...",Q3 2020,"November 5, 2020",5:00 PM ET
459,459,447,(HDSN),"Hudson Technologies, Inc.",Hudson Technologies Inc. (HDSN) CEO Brian Cole...,Q3 2020,"November 5, 2020",5:00 PM ET
460,460,448,(PACK),Ranpak Holdings Corp.,Ranpak Holdings Corp. (PACK) CEO Omar Asali on...,Q3 2020,"November 6, 2020",8:30 AM ET
461,461,449,(PFSW),"PFSweb, Inc.","PFSweb, Inc. (PFSW) CEO Mike Willoughby on Q3 ...",Q3 2020,"November 6, 2020",8:30 AM ET
462,462,450,(WSC),WillScot Mobile Mini Holdings Corp.,WillScot Mobile Mini Holdings' (WSC) CEO Brad ...,Q3 2020,"November 6, 2020",10:00 AM ET


In [17]:
Speech_table.head(10)

,Unnamed: 0,CC ID,Script ID,Participant,Speech,Section
0,0,0,0,Operator,['Good morning. My name is Dequan and I will b...,Speech
1,1,0,0,Jennifer Aitken,"['Thank you, Operator. Good morning, everyone,...",Speech
2,2,0,0,Mario Plourde,"['Thank you, Jennifer, and good morning, every...",Speech
3,3,0,0,Allan Hogg,"['Thank you, Mario, and good morning, everyone...",Speech
4,4,1,2,Operator,"['Ladies and gentlemen, thank you for standing...",Speech
5,5,1,2,Stefan Pettersson,"['Thank you, and good morning, everyone, and w...",Speech
6,6,1,2,Christian Fredrikson,"[""Yes. Thank you, and good morning, everyone, ...",Speech
7,7,2,4,Operator,"['Good day, ladies and gentlemen, and thank yo...",Speech
8,8,2,4,Unidentified Company Representative,"[""Thank you, operator. Before we begin, please...",Speech
9,9,2,4,John McGrath,"['Good morning. This is John McGrath, CEO of P...",Speech


In [18]:
QA_table.head(10)

,Unnamed: 0,CC ID,Script ID,Participant,Speech,Section
0,0,0,1,Operator,['[Operator Instructions]. Your first question...,Q&A
1,1,0,1,Hamir Patel,"[""Mario, could you speak to what sort of maybe...",Q&A
2,2,0,1,Mario Plourde,"[""Well, the target we are looking at right now...",Q&A
3,3,0,1,Allan Hogg,"[""Hamir, remember that our stated target is at...",Q&A
4,4,0,1,Hamir Patel,"['Okay. Great. Thanks for that. And Allan, cou...",Q&A
5,5,0,1,Allan Hogg,"[""Well, we have not completed yet our budget p...",Q&A
6,6,0,1,Hamir Patel,"['Okay. Thanks, Allan. And Charles, another qu...",Q&A
7,7,0,1,Charles Malo,"[""Okay. So Hamir, just maybe to complete on Al...",Q&A
8,8,0,1,Operator,['Your next question comes from Sean Steuart.'],Q&A
9,9,0,1,Sean Steuart,"[""Good morning, and thanks for all the detail ...",Q&A


In [19]:
Comp_part_table.head(10)

,Unnamed: 0,CC ID,Part ID,Participant,Position,Organization,Participant Type
0,0,0,0,Jennifer Aitken,"Director, IR",None,Internal
1,1,0,1,Mario Plourde,President & CEO,None,Internal
2,2,0,2,Allan Hogg,VP & CFO,None,Internal
3,3,0,3,Luc Langevin,"President & COO, Cascades Specialty Products ...",None,Internal
4,4,0,4,Charles Malo,"President & COO, Containerboard Packaging Group",None,Internal
5,5,0,5,Jean,David Tardif,None,Internal
6,6,1,11,Stefan Pettersson,"Head, IR",None,Internal
7,7,1,12,Christian Fredrikson,CEO & President,None,Internal
8,8,1,13,Per Sundqvist,CFO,None,Internal
9,9,2,17,John McGrath,Director,None,Internal


In [20]:
Conf_part_table.tail(10)

,Unnamed: 0,CC ID,Part ID,Participant,Position,Organization,Participant Type
2914,2914,459,4327,Lukas Hartwich,None,Green Street,External
2915,2915,459,4328,Stephen Grambling,None,Goldman Sachs,External
2916,2916,459,4329,Anthony Powell,None,Barclays,External
2917,2917,460,4335,Mark DeVries,None,Barclays,External
2918,2918,460,4336,Rick Shane,None,JPMorgan,External
2919,2919,460,4337,Sam Choe,None,Credit Suisse,External
2920,2920,460,4338,Tommy McJoynt,None,KBW,External
2921,2921,460,4339,Jack Micenko,None,SIG,External
2922,2922,460,4340,Geoffrey Dunn,None,Dowling & Partners,External
2923,2923,460,4341,Mark Hughes,None,Truist,External


In [21]:
#Company Table
company = company_info_table[['Tickers', 'Company']]
company.head(10)

,Tickers,Company
0,(CADNF),Cascades Inc.
1,(FGRRF),Fingerprint Cards AB (publ)
2,(PTVE),Pactiv Evergreen Inc.
3,(CNFR),"Conifer Holdings, Inc."
4,(EVOTF),Evotec SE
5,(BRP),"BRP Group, Inc."
6,(CCDBF),CCL Industries Inc.
7,(MCUJF),Medicure Inc.
8,(WLYYF),WELL Health Technologies Corp.
9,(HQI),"HireQuest, Inc."


In [22]:
#Conference Call Table
#get rid of duplicates

conf_call = company_info_table[['CC ID', 'Call Title', 'Tickers', 'Date', 'Time', 'Fiscal Quarter']]
conf_call.tail(10)

,CC ID,Call Title,Tickers,Date,Time,Fiscal Quarter
463,451,Fluidigm Corporation (FLDM) CEO Chris Linthwai...,(FLDM),"November 5, 2020",5:00 PM ET,Q3 2020
464,452,Potbelly Corporation (PBPB) CEO Bob Wright on ...,(PBPB),"November 5, 2020",5:00 PM ET,Q3 2020
465,453,Five Star Senior Living Inc. (FVE) CEO Katie P...,(FVE),"November 5, 2020",1:00 PM ET,Q1 2020
466,454,"IBI Group, Inc. (IBIBF) CEO Scott Stewart on Q...",(IBIBF),"November 6, 2020",8:30 AM ET,Q3 2020
467,455,"NN, Inc. (NNBR) CEO Warren Veltman on Q3 2020 ...",(NNBR),"November 6, 2020",9:00 AM ET,Q3 2020
468,456,Euronext N.V. (EUXTF) CEO Stéphane Boujnah on ...,(EUXTF),"November 6, 2020",3:00 AM ET,Q3 2020
469,457,Syros Pharmaceuticals' (SYRS) CEO Nancy Simoni...,(SYRS),"November 5, 2020",4:30 PM ET,Q3 2020
470,458,"Brigham Minerals, Inc. (MNRL) CEO Rob Roosa on...",(MNRL),"November 6, 2020",9:00 AM ET,Q3 2020
471,459,DiamondRock Hospitality Company (DRH) CEO Mark...,(DRH),"November 6, 2020",9:00 AM ET,Q3 2020
472,460,"NMI Holdings, Inc. (NMIH) CEO Claudia Merkle o...",(NMIH),"November 5, 2020",5:00 PM ET,Q3 2020


In [23]:
#Conference Participants

conf_part_1 = Comp_part_table[['CC ID','Part ID', 'Participant', 'Participant Type','Position', 'Organization']]
conf_part_2 = Conf_part_table[['CC ID','Part ID', 'Participant', 'Participant Type','Position', 'Organization']]


conf_part = pd.concat([conf_part_1, conf_part_2])
conf_part['Participant'] = conf_part['Participant'].str.strip()

conf_participants = conf_part[['Part ID', 'CC ID', 'Participant Type']]

conf_participants.tail(10)


,Part ID,CC ID,Participant Type
2914,4327,459,External
2915,4328,459,External
2916,4329,459,External
2917,4335,460,External
2918,4336,460,External
2919,4337,460,External
2920,4338,460,External
2921,4339,460,External
2922,4340,460,External
2923,4341,460,External


In [24]:
conf_part.head(10)

,CC ID,Part ID,Participant,Participant Type,Position,Organization
0,0,0,Jennifer Aitken,Internal,"Director, IR",None
1,0,1,Mario Plourde,Internal,President & CEO,None
2,0,2,Allan Hogg,Internal,VP & CFO,None
3,0,3,Luc Langevin,Internal,"President & COO, Cascades Specialty Products ...",None
4,0,4,Charles Malo,Internal,"President & COO, Containerboard Packaging Group",None
5,0,5,Jean,Internal,David Tardif,None
6,1,11,Stefan Pettersson,Internal,"Head, IR",None
7,1,12,Christian Fredrikson,Internal,CEO & President,None
8,1,13,Per Sundqvist,Internal,CFO,None
9,2,17,John McGrath,Internal,Director,None


In [25]:
#Create Internal Participant table
Int_part = conf_part[conf_part['Participant Type']=='Internal']
Internal_participants = Int_part[['Part ID','Participant','Position']]
Internal_participants.head(10)

,Part ID,Participant,Position
0,0,Jennifer Aitken,"Director, IR"
1,1,Mario Plourde,President & CEO
2,2,Allan Hogg,VP & CFO
3,3,Luc Langevin,"President & COO, Cascades Specialty Products ..."
4,4,Charles Malo,"President & COO, Containerboard Packaging Group"
5,5,Jean,David Tardif
6,11,Stefan Pettersson,"Head, IR"
7,12,Christian Fredrikson,CEO & President
8,13,Per Sundqvist,CFO
9,17,John McGrath,Director


In [26]:
#External Participants
Ext_part = conf_part[conf_part['Participant Type']=='External']
External_participants = Ext_part[['Part ID','Participant','Organization']]
External_participants.head(10)

,Part ID,Participant,Organization
0,6,Hamir Patel,CIBC Capital Markets
1,7,Sean Steuart,TD Securities
2,8,Mark Wilde,BMO Capital Markets
3,9,Zachary Evershed,National Bank Financial
4,10,Paul Quinn,RBC Capital Markets
5,14,Francois Bouvignies,UBS Investment Bank
6,15,Rob Sanders,Deutsche Bank
7,16,Viktor Westman,Redeye AB
8,19,Anthony Pettinari,Citigroup
9,20,Thomas Digenan,Robert W. Baird & Co.


In [27]:
#Presentation Details

pres = pd.concat([Speech_table, QA_table])
pres['Participant'] = pres['Participant'].str.strip()

pres_table = pd.merge(pres,conf_part,how='inner',left_on=['CC ID','Participant'],right_on =['CC ID','Participant'])
pres_table.head(20)


,Unnamed: 0,CC ID,Script ID,Participant,Speech,Section,Part ID,Participant Type,Position,Organization
0,1,0,0,Jennifer Aitken,"['Thank you, Operator. Good morning, everyone,...",Speech,0,Internal,"Director, IR",None
1,2,0,0,Mario Plourde,"['Thank you, Jennifer, and good morning, every...",Speech,1,Internal,President & CEO,None
2,2,0,1,Mario Plourde,"[""Well, the target we are looking at right now...",Q&A,1,Internal,President & CEO,None
3,10,0,1,Mario Plourde,"['Well, I will let Jean-David explain the dyna...",Q&A,1,Internal,President & CEO,None
4,25,0,1,Mario Plourde,"[""Mark, it's Mario. Basically right now, we se...",Q&A,1,Internal,President & CEO,None
5,29,0,1,Mario Plourde,"[""It's lower than that, but we were in the hig...",Q&A,1,Internal,President & CEO,None
6,43,0,1,Mario Plourde,"[""I would see the same progression, Paul. We'v...",Q&A,1,Internal,President & CEO,None
7,45,0,1,Mario Plourde,"['Thank you, everyone, for being on the line t...",Q&A,1,Internal,President & CEO,None
8,3,0,0,Allan Hogg,"['Thank you, Mario, and good morning, everyone...",Speech,2,Internal,VP & CFO,None
9,3,0,1,Allan Hogg,"[""Hamir, remember that our stated target is at...",Q&A,2,Internal,VP & CFO,None


In [28]:
#Presentation Type
pres_clean = pres[['CC ID','Script ID','Section']]
Presentation_type = pres_clean.sort_values('Script ID')

Presentation_type.head(50)

,CC ID,Script ID,Section
0,0,0,Speech
1,0,0,Speech
2,0,0,Speech
3,0,0,Speech
26,0,1,Q&A
27,0,1,Q&A
28,0,1,Q&A
29,0,1,Q&A
30,0,1,Q&A
31,0,1,Q&A


In [30]:
Call_Script = pres_table[['Script ID','Section','Speech','Part ID']]
Call_Script.head(20)

,Script ID,Section,Speech,Part ID
0,0,Speech,"['Thank you, Operator. Good morning, everyone,...",0
1,0,Speech,"['Thank you, Jennifer, and good morning, every...",1
2,1,Q&A,"[""Well, the target we are looking at right now...",1
3,1,Q&A,"['Well, I will let Jean-David explain the dyna...",1
4,1,Q&A,"[""Mark, it's Mario. Basically right now, we se...",1
5,1,Q&A,"[""It's lower than that, but we were in the hig...",1
6,1,Q&A,"[""I would see the same progression, Paul. We'v...",1
7,1,Q&A,"['Thank you, everyone, for being on the line t...",1
8,0,Speech,"['Thank you, Mario, and good morning, everyone...",2
9,1,Q&A,"[""Hamir, remember that our stated target is at...",2


# Get to My SQL Code 

In [33]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [31]:
import pymysql
import pandas as pd
from sqlalchemy import create_engine

In [34]:
#This is your personal info from your my sql workbench
hostname="localhost"
dbname="data_mgmt_final"
uname="root"
pwd="Basketball11!"

In [35]:
# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(host=hostname, db=dbname, user=uname, pw=pwd))

In [36]:
# Convert dataframe to sql table                                   
company.to_sql('Company', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Company' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [37]:
# Convert dataframe to sql table                                   
conf_call.to_sql('Conference_Call', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Conference_Call' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [38]:
# Convert dataframe to sql table                                   
conf_participants.to_sql('Conf_Participants', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Conf_Participants' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [39]:
# Convert dataframe to sql table                                   
Internal_participants.to_sql('Int_Participants', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Int_Participants' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [40]:
# Convert dataframe to sql table                                   
External_participants.to_sql('Ext_Participants', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Ext_Participants' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [41]:
# Convert dataframe to sql table                                   
Presentation_type.to_sql('Pres_Type', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Pres_Type' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [42]:
# Convert dataframe to sql table                                   
Call_Script.to_sql('Call_Script', engine, index=False)

C:\Users\saman\anaconda3\lib\site-packages\pandas\io\sql.py:1333: UserWarning: The provided table name 'Call_Script' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


In [ ]:
# Convert dataframe to sql table                                   
df3.to_sql('users', engine, index=False)